# Define funcoes

## freqTbl function

In [ ]:
import pandas as pd
from tabulate import tabulate
import matplotlib.pyplot as plt

def freqTbl(dataframe, nome_coluna, titulo_tabela, nome_indice="Categoria", salvar_imagem=False, nome_arquivo=None):
    # 1. Contagem de frequência
    frequencia = dataframe[nome_coluna].value_counts(dropna=False).sort_index()

    # 2. Cálculo da porcentagem
    porcentagem = dataframe[nome_coluna].value_counts(normalize=True, dropna=False).sort_index() * 100

    # 3. Cálculo da porcentagem acumulada
    porcentagem_acumulada = porcentagem.cumsum()

    # 4. Combinar em uma tabela (DataFrame)
    tabela_descritiva = pd.DataFrame({
        'Frequência': frequencia,
        'Porcentagem (%)': porcentagem.round(2).astype(str) + '%',
        'Porcentagem Acumulada (%)': porcentagem_acumulada.round(2).astype(str) + '%'
    })
    tabela_descritiva.index.name = nome_indice

    # 5. Adicionar linha de Totais
    total_frequencia = tabela_descritiva['Frequência'].sum()
    # A porcentagem total é 100% e a acumulada no final também é 100%
    # Para fins de exibição, mantemos a string formatada
    tabela_descritiva.loc['Total'] = [total_frequencia, '100.00%', '100.00%']

    # 6. Gerar tabela formatada com tabulate
    tabela_formatada = tabulate(
        tabela_descritiva,
        headers='keys',
        tablefmt='grid',  # Formato mais visual para imagem
        showindex=True,
        stralign="center",
        numalign="center"
    )

    # 7. Exibir no console (opcional)
    print(f"\n{titulo_tabela}")
    print(tabela_formatada)
    print("-" * 60) # Ajustado para a nova largura da tabela

    # 8. Gerar e salvar imagem (se solicitado)
    if salvar_imagem:
        # Criar uma cópia da tabela para a imagem sem o tipo de dado string nos totais
        # para melhor formatação visual no matplotlib, se necessário, ou usar a tabela_descritiva diretamente
        # Neste caso, como já formatamos como string com '%', podemos usar diretamente.

        # Ajustar o tamanho da figura com base no número de linhas e colunas
        num_rows = len(tabela_descritiva)
        num_cols = len(tabela_descritiva.columns)
        fig_height = max(4, num_rows * 0.5)  # Altura mínima de 4, aumenta com as linhas
        fig_width = max(10, num_cols * 2.5) # Largura mínima de 10, aumenta com as colunas

        plt.figure(figsize=(fig_width, fig_height))
        plt.axis('off')  # Desativa eixos
        plt.title(titulo_tabela, fontsize=14, pad=20) # Aumentei um pouco o fontsize do título

        # Adiciona a tabela à figura
        # Matplotlib table é um pouco diferente de tabulate, então precisamos passar os dados de forma explícita
        table_for_plot = plt.table(
            cellText=tabela_descritiva.values,
            colLabels=tabela_descritiva.columns,
            rowLabels=tabela_descritiva.index,
            loc='center',
            cellLoc='center',
            colColours=['#f0f0f0'] * len(tabela_descritiva.columns)  # Cor de fundo do cabeçalho
        )
        table_for_plot.auto_set_font_size(False)
        table_for_plot.set_fontsize(10)
        table_for_plot.scale(1.2, 1.2) # Ajusta a escala da tabela se necessário

        plt.tight_layout(pad=1.0) # Adiciona um pouco de padding

        # Salva a imagem
        if nome_arquivo is None:
            nome_arquivo = f"tabela_{nome_coluna}.png"
        plt.savefig(nome_arquivo, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"Tabela salva como '{nome_arquivo}'")

    return tabela_descritiva